In [1]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
import requests
from dotenv import load_dotenv

load_dotenv() 


llm = ChatOpenAI(model="gpt-4o-mini")

In [2]:
prompt = ChatPromptTemplate.from_template("""O usuário está buscando informações sobre o clima de um determinado lugar, extraia da mensagem dele apenas o lugar de interesse.
                                          Mensagem: {mensagem}""")

In [3]:
cidade_latlon = {"Guaratuba":[-25.88055638531835, -48.57584844726812], "São Paulo":[-23.552095602314147, -46.637115778597305]}

In [4]:
def temperatura(cidade:str):

    lista = cidade_latlon.get(cidade)
    lat,lon = lista[0],lista[1]
    response = requests.get(
    f'https://api.open-meteo.com/v1/forecast?latitude={lat}&longitude={lon}&current=temperature_2m&hourly=temperature_2m',
)
    return response.json()

In [5]:
prompt_assistente = ChatPromptTemplate.from_template("""Você é um assistente pessoal que tem o objetivo de facilitar a vida do seu usuário.
                                          Responda sempre com leveza e dê informações que sejam úteis.
                                          Clima: {clima}
                                          Local: {local}
                                          Mensagem: {mensagem}""")

In [6]:
extrair_local = prompt | llm | StrOutputParser()
conversa = prompt_assistente | llm | StrOutputParser()

def chain_completa(mensagem: str):
    local = extrair_local.invoke(mensagem)
    response = conversa.invoke({"clima": temperatura(local), "local": local,
                                "mensagem":mensagem})
    return response

print(chain_completa("Como está o clima em Guaratuba? Preciso de um casaco para sair agora?"))


Olá! Em Guaratuba, neste momento, a temperatura está em torno de **27°C**. O clima parece estar bem agradável, mas se você é sensível a temperaturas mais frescas, um casaco leve pode ser uma boa ideia, especialmente se você planeja ficar ao ar livre por um tempo. Se precisar de mais alguma coisa, é só avisar! 😊


In [7]:
print(chain_completa("Estou indo para São Paulo hoje, aqui onde estou está bem frio, continuo com as blusas ou devo mudar?"))

Oi! Que delícia saber que você está indo para São Paulo! Hoje, a temperatura está bem agradável por lá, em torno de 24.4°C. Então, é uma boa ideia dar uma leveza no seu look! Se você gosta de conforto, uma blusa leve pode ser uma ótima escolha. Assim, você fica preparado para qualquer mudança de temperatura durante o dia. Aproveite sua viagem! 🌞✨
